In [1]:
from sklearn.linear_model import LinearRegression
import pandas as pd

df = pd.read_csv('World-Stock-Prices-Dataset.csv')


df["Country"].unique()


array(['usa', 'germany', 'japan', 'switzerland', 'canada', 'netherlands',
       'france'], dtype=object)

In [2]:
# Explore the dataset structure
print("Dataset shape:", df.shape)
print("\nColumn names:")
print(df.columns.tolist())
print("\nFirst few rows:")
print(df.head())
print("\nData types:")
print(df.dtypes)
print("\nBasic statistics:")
print(df.describe())


Dataset shape: (310122, 13)

Column names:
['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Brand_Name', 'Ticker', 'Industry_Tag', 'Country', 'Dividends', 'Stock Splits', 'Capital Gains']

First few rows:
                        Date        Open        High         Low       Close  \
0  2025-07-03 00:00:00-04:00    6.630000    6.740000    6.615000    6.640000   
1  2025-07-03 00:00:00-04:00  106.750000  108.370003  106.330101  107.339996   
2  2025-07-03 00:00:00-04:00  122.629997  123.050003  121.550003  121.930000   
3  2025-07-03 00:00:00-04:00  221.705002  224.009995  221.360001  223.410004   
4  2025-07-03 00:00:00-04:00  212.145004  214.649994  211.810104  213.550003   

       Volume Brand_Name Ticker Industry_Tag  Country  Dividends  \
0   4209664.0    peloton   PTON      fitness      usa        0.0   
1    560190.0      crocs   CROX     footwear      usa        0.0   
2     36600.0     adidas  ADDYY      apparel  germany        0.0   
3  29295154.0     amazon   AMZN   e-com

4) which company had the most volatile stock in terms of daily range (high to low gap)

In [3]:
df['Volatility (%)'] = (df['High'] - df['Low']) / df['Low'] * 100
max_vol_row = df.loc[df['Volatility (%)'].idxmax()]
print(max_vol_row)

Date              2019-06-10 00:00:00-04:00
Open                             590.928222
High                             600.041075
Low                               67.861498
Close                             92.097748
Volume                                200.0
Brand_Name                             puma
Ticker                                PMMAF
Industry_Tag                        apparel
Country                             germany
Dividends                               0.0
Stock Splits                           10.0
Capital Gains                           NaN
Volatility (%)                   784.214303
Name: 94646, dtype: object


5) identify the companies that paid dividends at some point in the dataset
6) which company's stock showed the greatest upwards trend over time

In [6]:
df['Date'] = pd.to_datetime(df['Date'], utc=True)
results = []

for company, group in df.groupby('Brand_Name'):
  group = group.sort_values(by='Date')
  if len(group) < 2:
    continue
  x = group['Date'].map(pd.Timestamp.toordinal).values.reshape(-1, 1)
  y = group['Close'].values

  model = LinearRegression().fit(x, y)

  start_date = x[0, 0]
  end_date = x[-1, 0]
  start_pred = model.predict([[start_date]])[0]
  end_pred = model.predict([[end_date]])[0]

  increase = (end_pred - start_pred) / start_pred * 100

  results.append({
    'Brand_Name': company,
    'Start_Date': group['Date'].iloc[0],
    'End_Date': group['Date'].iloc[-1],
    'Start_Price': start_pred,
    'End_Price': end_pred,
    'Increase': increase,
  })

results_df = pd.DataFrame(results)

max_increase = results_df.loc[results_df['Increase'].idxmax()]
print(max_increase)

Brand_Name                      unilever
Start_Date     2000-01-03 05:00:00+00:00
End_Date       2025-07-03 04:00:00+00:00
Start_Price                     0.440314
End_Price                      57.123383
Increase                    12873.335937
Name: 57, dtype: object
